In [1]:
import os.path as osp

import gym
import gym.spaces as spaces
import numpy as np

import habitat
import habitat_baselines.utils.gym_definitions as habitat_gym
from habitat.core.embodied_task import Measure
from habitat.core.registry import registry
from habitat.core.simulator import Sensor, SensorTypes
from habitat.tasks.rearrange.rearrange_sensors import RearrangeReward
from habitat.tasks.rearrange.sub_tasks.pick_task import RearrangePickTaskV1
from habitat.utils.visualizations.utils import observations_to_image
from habitat_baselines.utils.render_wrapper import overlay_frame
from habitat_sim.utils import viz_utils as vut

In [2]:
def insert_render_options(config):
    config.defrost()
    config.SIMULATOR.THIRD_RGB_SENSOR.WIDTH = 512
    config.SIMULATOR.THIRD_RGB_SENSOR.HEIGHT = 512
    config.SIMULATOR.AGENT_0.SENSORS.append("THIRD_RGB_SENSOR")
    config.freeze()
    return config

In [3]:
@registry.register_task(name="RearrangeStackTask-v1")
class RearrangeStackTaskV1(RearrangePickTaskV1):
    """
    Rearrange Stack Task with Fetch robot interacting with objects and environment.
    """
    
    def step(self, action, episode):
        action_args = action["action_args"]
        obs = super().step(action=action, episode=episode)
        return obs

@registry.register_measure
class TowerHeightMeasure(Measure):
    """
    Tracks the maximum 'y' coordinate of any object in the scene.
    """
    
    cls_uuid: str = "tower_height"
        
    def __init__(self, sim, config, *args, **kwargs):
        self._sim = sim
        self._config = config
        super().__init__(**kwargs)
        
        
    @staticmethod
    def _get_uuid(*args, **kwargs):
        return TowerHeightMeasure.cls_uuid
    
    def reset_metric(self, *args, episode, **kwargs):
        self.update_metric(*args, episode=episode, **kwargs)

    def update_metric(self, *args, episode, **kwargs):
        # Get positions of target objects
        # TODO: in the future, need to account for the fact that some objects may not be part of the tower
        # and not held by the robot?
        rom = self._sim.get_rigid_object_manager()
        scene_pos = self._sim.get_scene_pos()
        self.target_pos = np.array(
            [
                scene_pos[
                    self._sim.scene_obj_ids.index(
                        rom.get_object_by_handle(t_handle).object_id
                    )
                ]
                for t_handle, _ in self._sim.ep_info["targets"].items()
            ]
        )
        
        self._metric = 0
        obj_ids = np.array(self._sim.scene_obj_ids)
        target_pos_arr = np.array(self.target_pos)
        
        # Assumption: only one obj can be gripped
        if self._sim.grasp_mgr._snapped_obj_id is not None:
            # Remove gripped obj from list of objs to measure
            target_pos_arr = np.delete(target_pos_arr,
                                        np.where(obj_ids==self._sim.grasp_mgr._snapped_obj_id)[0][0],
                                        0)
        if len(target_pos_arr) > 0:
            self._metric = target_pos_arr[:,1].max()
    

@registry.register_measure
class StackRewardMeasure(Measure):
    """
    Determines agent reward based on the change in height of stacked tower.
    """
    cls_uuid: str = "stack_reward"
        
    def __init__(self, sim, config, *args, **kwargs):
        self._sim = sim
        self._config = config
        self.prev_max_y = None
        super().__init__(**kwargs)
        
    @staticmethod
    def _get_uuid(*args, **kwargs):
        return StackRewardMeasure.cls_uuid
    
    def reset_metric(self, *args, task, episode, **kwargs):
        task.measurements.check_measure_dependencies(
            self.uuid,
            [
                TowerHeightMeasure.cls_uuid,
            ],
        )
        self.update_metric(*args, task=task, episode=episode, **kwargs)
        
    def update_metric(self, *args, task, episode, **kwargs):
        curr_max_y = task.measurements.measures[
                TowerHeightMeasure.cls_uuid
        ].get_metric()
            
        if self.prev_max_y is not None:
            self._metric = curr_max_y - self.prev_max_y
        else:
            self._metric = 0
        
        self.prev_max_y = curr_max_y


In [4]:
cfg_txt = """
ENVIRONMENT:
    MAX_EPISODE_STEPS: 200
DATASET:
    TYPE: RearrangeDataset-v0
    SPLIT: train
    DATA_PATH: data/datasets/rearrange_pick/replica_cad/v0/rearrange_pick_replica_cad_v0/pick.json.gz
    SCENES_DIR: "data/replica_cad/"
TASK:
    TYPE: RearrangePickTask-v0
    MAX_COLLISIONS: -1.0
    COUNT_OBJ_COLLISIONS: True
    COUNT_ROBOT_OBJ_COLLS: False
    DESIRED_RESTING_POSITION: [0.5, 0.0, 1.0]

    # In radians
    BASE_ANGLE_NOISE: 0.15
    BASE_NOISE: 0.05
    CONSTRAINT_VIOLATION_ENDS_EPISODE: True
    FORCE_REGENERATE: False

    # Measurements for composite tasks.
    REWARD_MEASUREMENT: "rearrangepick_reward"
    SUCCESS_MEASUREMENT: "rearrangepick_success"

    # If true, does not care about navigability or collisions with objects when spawning
    # robot
    EASY_INIT: False

    TARGET_START_SENSOR:
        TYPE: "TargetStartSensor"
        GOAL_FORMAT: "CARTESIAN"
        DIMENSIONALITY: 3
    GOAL_SENSOR:
        TYPE: "GoalSensor"
        GOAL_FORMAT: "CARTESIAN"
        DIMENSIONALITY: 3
    ABS_TARGET_START_SENSOR:
        TYPE: "AbsTargetStartSensor"
        GOAL_FORMAT: "CARTESIAN"
        DIMENSIONALITY: 3
    ABS_GOAL_SENSOR:
        TYPE: "AbsGoalSensor"
        GOAL_FORMAT: "CARTESIAN"
        DIMENSIONALITY: 3
    JOINT_SENSOR:
        TYPE: "JointSensor"
        DIMENSIONALITY: 7
    END_EFFECTOR_SENSOR:
        TYPE: "EEPositionSensor"
    IS_HOLDING_SENSOR:
        TYPE: "IsHoldingSensor"
    RELATIVE_RESTING_POS_SENSOR:
        TYPE: "RelativeRestingPositionSensor"
    SENSORS: ["TARGET_START_SENSOR", "JOINT_SENSOR", "IS_HOLDING_SENSOR", "END_EFFECTOR_SENSOR", "RELATIVE_RESTING_POS_SENSOR"]
    ROBOT_FORCE:
        TYPE: "RobotForce"
        MIN_FORCE: 20.0
    EXCESSIVE_FORCE_SHOULD_END:
        TYPE: "ForceTerminate"
        MAX_ACCUM_FORCE: 5000.0
    ROBOT_COLLS:
        TYPE: "RobotCollisions"
    OBJECT_TO_GOAL_DISTANCE:
        TYPE: "ObjectToGoalDistance"
    END_EFFECTOR_TO_OBJECT_DISTANCE:
        TYPE: "EndEffectorToObjectDistance"
    END_EFFECTOR_TO_REST_DISTANCE:
        TYPE: "EndEffectorToRestDistance"
    PICK_REWARD:
        TYPE: "RearrangePickReward"
        DIST_REWARD: 20.0
        SUCC_REWARD: 10.0
        PICK_REWARD: 20.0
        DROP_PEN: 5.0
        WRONG_PICK_PEN: 5.0
        USE_DIFF: True
        DROP_OBJ_SHOULD_END: False
        WRONG_PICK_SHOULD_END: False

        # General Rearrange Reward config
        CONSTRAINT_VIOLATE_PEN: 10.0
        FORCE_PEN: 0.001
        MAX_FORCE_PEN: 1.0
        FORCE_END_PEN: 10.0


    PICK_SUCCESS:
        TYPE: "RearrangePickSuccess"
        SUCC_THRESH: 0.15

    TOWER_HEIGHT:
        TYPE: "TowerHeightMeasure"

    STACK_REWARD:
        TYPE: "StackRewardMeasure"

    MEASUREMENTS:
        - "OBJECT_TO_GOAL_DISTANCE"
        - "ROBOT_FORCE"
        - "EXCESSIVE_FORCE_SHOULD_END"
        - "ROBOT_COLLS"
        - "END_EFFECTOR_TO_REST_DISTANCE"
        - "END_EFFECTOR_TO_OBJECT_DISTANCE"
        - "PICK_SUCCESS"
        - "PICK_REWARD"
        - "TOWER_HEIGHT"
        - "STACK_REWARD"
    ACTIONS:
        ARM_ACTION:
            TYPE: "ArmAction"
            ARM_CONTROLLER: "ArmRelPosAction"
            GRIP_CONTROLLER: "MagicGraspAction"
            ARM_JOINT_DIMENSIONALITY: 7
            GRASP_THRESH_DIST: 0.15
            DISABLE_GRIP: False
            DELTA_POS_LIMIT: 0.0125
            EE_CTRL_LIM: 0.015
    POSSIBLE_ACTIONS:
        - ARM_ACTION

SIMULATOR:
    ACTION_SPACE_CONFIG: v0
    AGENTS: ['AGENT_0']
    DEBUG_RENDER: False
    ROBOT_JOINT_START_NOISE: 0.0
    AGENT_0:
        HEIGHT: 1.5
        IS_SET_START_STATE: False
        RADIUS: 0.1
        SENSORS: ['HEAD_RGB_SENSOR', 'HEAD_DEPTH_SENSOR', 'ARM_RGB_SENSOR', 'ARM_DEPTH_SENSOR']
        START_POSITION: [0, 0, 0]
        START_ROTATION: [0, 0, 0, 1]
    HEAD_RGB_SENSOR:
        WIDTH: 128
        HEIGHT: 128
    HEAD_DEPTH_SENSOR:
        WIDTH: 128
        HEIGHT: 128
        MIN_DEPTH: 0.0
        MAX_DEPTH: 10.0
        NORMALIZE_DEPTH: True
    ARM_DEPTH_SENSOR:
        HEIGHT: 128
        MAX_DEPTH: 10.0
        MIN_DEPTH: 0.0
        NORMALIZE_DEPTH: True
        WIDTH: 128
    ARM_RGB_SENSOR:
        HEIGHT: 128
        WIDTH: 128

    # Agent setup
    ARM_REST: [0.6, 0.0, 0.9]
    CTRL_FREQ: 120.0
    AC_FREQ_RATIO: 4
    ROBOT_URDF: ./data/robots/hab_fetch/robots/hab_fetch.urdf
    ROBOT_TYPE: "FetchRobot"
    FORWARD_STEP_SIZE: 0.25

    # Grasping
    HOLD_THRESH: 0.09
    GRASP_IMPULSE: 1000.0

    DEFAULT_AGENT_ID: 0
    HABITAT_SIM_V0:
        ALLOW_SLIDING: True
        ENABLE_PHYSICS: True
        GPU_DEVICE_ID: 0
        GPU_GPU: False
        PHYSICS_CONFIG_FILE: ./data/default.physics_config.json
    SEED: 100
    SEMANTIC_SENSOR:
        HEIGHT: 480
        HFOV: 90
        ORIENTATION: [0.0, 0.0, 0.0]
        POSITION: [0, 1.25, 0]
        TYPE: HabitatSimSemanticSensor
        WIDTH: 640
    TILT_ANGLE: 15
    TURN_ANGLE: 10
    TYPE: RearrangeSim-v0
"""
nav_pick_cfg_path = "data/stack.yaml"
with open(nav_pick_cfg_path, "w") as f:
    f.write(cfg_txt)

In [5]:
with habitat.Env(
    config=insert_render_options(habitat.get_config(nav_pick_cfg_path))
) as env:
    env.reset()

    print("Agent acting inside environment.")
    count_steps = 0
    # To save the video
    video_file_path = "data/rearrange_stack_test.mp4"
    video_writer = vut.get_fast_video_writer(video_file_path, fps=30)

    while not env.episode_over:
        action = env.action_space.sample()
        observations = env.step(action)  # noqa: F841
        info = env.get_metrics()

        render_obs = observations_to_image(observations, info)
        render_obs = overlay_frame(render_obs, info)

        video_writer.append_data(render_obs)

        count_steps += 1
    print("Episode finished after {} steps.".format(count_steps))

    video_writer.close()
    if vut.is_notebook():
        vut.display_video(video_file_path)

2021-12-20 13:55:05,318 Initializing dataset RearrangeDataset-v0
2021-12-20 13:55:05,348 initializing sim RearrangeSim-v0
2021-12-20 13:55:07,230 Initializing task RearrangePickTask-v0


Agent acting inside environment.
Episode finished after 200 steps.
